In [1]:
import seaborn as sns

import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from sklearn.linear_model import LogisticRegression

from sklearn import set_config
set_config(transform_output="pandas")

# Set random seed 
RSEED = 42

warnings.filterwarnings("ignore")

In [52]:
df = pd.read_csv("data/data.csv")

In [3]:
df

Rescue_ID Date_TimeCaught     Researcher    CaptureSite  \
0      2000_RE_0060      2000-12-22  Researcher_25  CaptureSite_0   
1      2001_RE_0187      2001-10-28   Researcher_6  CaptureSite_0   
2      2001_RE_0197      2001-11-01   Researcher_6  CaptureSite_0   
3      2002_RE_0031      2002-03-11  Researcher_32  CaptureSite_0   
4      2002_RE_0118      2002-08-08  Researcher_25  CaptureSite_0   
...             ...             ...            ...            ...   
18057  2018_RE_1511      2018-12-18  Researcher_30  CaptureSite_9   
18058  2018_RE_1514      2018-12-18  Researcher_30  CaptureSite_9   
18059  2018_RE_1532      2018-12-24  Researcher_30  CaptureSite_9   
18060  2018_RE_1533      2018-12-24  Researcher_30  CaptureSite_9   
18061  2018_RE_1550      2018-12-28  Researcher_30  CaptureSite_9   

      ForagingGround CaptureMethod       Fisher  \
0              Ocean           Net  Fisher_1072   
1              Ocean           Net   Fisher_520   
2              Ocean           Net  Fisher_1669   
3              Ocean           Net  Fisher_1798   
4              Ocean       Beached  Fisher_1918   
...              ...           ...          ...   
18057          Ocean           Net   Fisher_569   
18058          Ocean           Net   Fisher_125   
18059          Ocean           Net  Fisher_1343   
18060          Ocean           Net  Fisher_1551   
18061          Ocean           Net  Fisher_1551   

                             LandingSite    Species           Tag_1  ...  \
0      LandingSite_CaptureSiteCategory_2  Species_6         CC00147  ...   
1      LandingSite_CaptureSiteCategory_2  Species_6            W442  ...   
2      LandingSite_CaptureSiteCategory_2  Species_5          KE0376  ...   
3      LandingSite_CaptureSiteCategory_2  Species_6         CC00302  ...   
4      LandingSite_CaptureSiteCategory_2  Species_5  NotTagged_0113  ...   
...                                  ...        ...             ...  ...   
18057  LandingSite_CaptureSiteCategory_1  Species_5         KES1828  ...   
18058  LandingSite_CaptureSiteCategory_1  Species_6         KES0563  ...   
18059  LandingSite_CaptureSiteCategory_1  Species_5         KES1833  ...   
18060  LandingSite_CaptureSiteCategory_1  Species_5         KES1831  ...   
18061  LandingSite_CaptureSiteCategory_1  Species_6         KES1432  ...   

      Lost_Tags T_Number CCL_cm  CCW_cm  Weight_Kg      Sex  \
0           NaN      NaN  64.70   62.60        NaN  Unknown   
1           NaN      NaN  35.85   31.35        NaN  Unknown   
2           NaN      NaN  51.80   49.20        NaN  Unknown   
3           NaN      NaN  60.50   59.00        NaN  Unknown   
4           NaN      NaN  34.70   33.00        NaN  Unknown   
...         ...      ...    ...     ...        ...      ...   
18057       NaN      NaN  57.13   50.57      21.09  Unknown   
18058   KES0416      NaN  42.07   38.37       9.02  Unknown   
18059       NaN      NaN  57.20   52.30        NaN  Unknown   
18060       NaN      NaN  51.90   48.50        NaN  Unknown   
18061       NaN      NaN  34.60   31.20       4.29  Unknown   

                                   TurtleCharacteristics    Status  \
0                                 algae at rear of shell  Released   
1      multiple b's on front flippers&  a lot of alga...  Released   
2                                                  clean  Released   
3      1 b 3 CS+ calcerous algae at rear end of shell...  Released   
4      very lively+ right eye is hanging out + swolle...  Released   
...                                                  ...       ...   
18057                 White calcareous algae on carapace  Released   
18058  Calcareous + green algae on carapace\nBarnacle...  Released   
18059                                       Clean turtle  Released   
18060  Green algae on carapace\ntip of left supra mis...  Released   
18061  Thick pink patches of calcareous algae on cara...  Released   

          ReleaseSite Date_TimeRelease  
0      ReleaseSi

In [7]:
variable_def = pd.read_csv("data/variable_definitions.csv",encoding='latin-1')
pd.set_option('display.max_colwidth', None)
variable_def

Variables  \
0                Rescue_ID   
1          Date_TimeCaught   
2               Researcher   
3             Capture Site   
4          Foraging Ground   
5           Capture Method   
6                   Fisher   
7             Landing_Site   
8                  Species   
9                    Tag_1   
10                   Tag_2   
11               Lost_Tags   
12                T_Number   
13                  CCL_cm   
14                  CCW_cm   
15               Weight_Kg   
16                     Sex   
17  Turtle Characteristics   
18                  Status   
19            Release Site   
20            Date_Release   
21    Release_Admiss_Notes   
22          SpecialRemarks   

                                                                                                                                                                                                                                                                                     Description  
0                                                                                                                                       Its an individual bycatch incidence identity number. The numbers are consecutive, for each year e.g.  2018_RE_0732 means rescue number 732 in year 2018  
1                                                                                                                                                                                                                                                   Date the turtle is captured e.g. 06/01/2018   
2                                                                                                                                                                                                                                       Name of bycatch officer(s) involved with specific rescue  
3                                                                                                                                                                                                                                     Area where turtle was captured, as reported by the fisher.  
4                                                                                  General area of ocean area where turtle was captured. The assumption is that the turtle was foraging where it was captured. The foraging area is broadly classified either as the open ocean or creek section  
5                                                                                                                                                                                                                                   Fishing gear or method used by fishers to capture the turtle  
6                                                                                                                                                                                                                                                     Name of the fisher who captured the turtle  
7                                                                                                                                                                                                                                                       Section of beach where turtle is landed.  
8                                                                                                                                                                                                                              Species of turtle (e.g. green turtle, hawksbill, loggerhead etc.)  
9   Individual / unique number used to identify a turtle. Each turtle that is captured is tagged. Formats of tag numbers have been changed over years, although turtles retain their old tags even when recaptured (unless the tags are about to fall or are too tight, then they are replaced).  
10                                                                                             

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18062 entries, 0 to 18061
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Rescue_ID              18062 non-null  object 
 1   Date_TimeCaught        18062 non-null  object 
 2   Researcher             18062 non-null  object 
 3   CaptureSite            18062 non-null  object 
 4   ForagingGround         18062 non-null  object 
 5   CaptureMethod          18062 non-null  object 
 6   Fisher                 18062 non-null  object 
 7   LandingSite            18062 non-null  object 
 8   Species                18062 non-null  object 
 9   Tag_1                  17937 non-null  object 
 10  Tag_2                  4911 non-null   object 
 11  Lost_Tags              925 non-null    object 
 12  T_Number               38 non-null     object 
 13  CCL_cm                 18038 non-null  float64
 14  CCW_cm                 18035 non-null  float64
 15  We

In [60]:
df.isnull().sum()

Rescue_ID                    0
Date_TimeCaught              0
Researcher                   0
CaptureSite                  0
ForagingGround               0
CaptureMethod                0
Fisher                       0
LandingSite                  0
Species                      0
Tag_1                      125
Tag_2                    13151
Lost_Tags                17137
T_Number                 18024
CCL_cm                       0
CCW_cm                      27
Weight_Kg                 5409
Sex                       4330
TurtleCharacteristics       52
Status                    3633
ReleaseSite                 75
Date_TimeRelease          6108
dtype: int64

In [15]:
# import missingno
import missingno as msno

In [59]:
print((df==0).sum())

Rescue_ID                0
Date_TimeCaught          0
Researcher               0
CaptureSite              0
ForagingGround           0
CaptureMethod            0
Fisher                   0
LandingSite              0
Species                  0
Tag_1                    0
Tag_2                    0
Lost_Tags                0
T_Number                 0
CCL_cm                   0
CCW_cm                   0
Weight_Kg                0
Sex                      0
TurtleCharacteristics    0
Status                   0
ReleaseSite              0
Date_TimeRelease         0
dtype: int64


In [ ]:
# import missingno
import missingno as msno

Deleting Tag_1 and Lost_tags column:

In [61]:
del df['Tag_1']

In [63]:
del df['Lost_Tags']

Replacing NaN Values in Tag_2 column with zeros:

In [64]:

df['Tag_2'].fillna(0, inplace=True)


Replacing string values in Tag_2 column (which represent a large turtle) with 1:

In [65]:

df['Tag_2'] = df['Tag_2'].replace(to_replace='.*', value=1, regex=True)

df['Tag_2'].unique()


array([0, 1], dtype=int64)

In [79]:
df['Tag_2'].value_counts()

Tag_2
0    13151
1     4911
Name: count, dtype: int64

Replacing NaN Values in T_Number column with zeros:

In [70]:

df['T_Number'].fillna(0, inplace=True) 

Replacing string values in T-Number column (which represent a visit in a rehabilitation facility ) with 1:

In [71]:

df['T_Number'] = df['T_Number'].replace(to_replace='.*', value=1, regex=True)

df['T_Number'].unique()

array([0., 1.])

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18062 entries, 0 to 18061
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Rescue_ID              18062 non-null  object 
 1   Date_TimeCaught        18062 non-null  object 
 2   Researcher             18062 non-null  object 
 3   CaptureSite            18062 non-null  object 
 4   ForagingGround         18062 non-null  object 
 5   CaptureMethod          18062 non-null  object 
 6   Fisher                 18062 non-null  object 
 7   LandingSite            18062 non-null  object 
 8   Species                18062 non-null  object 
 9   Tag_2                  18062 non-null  int64  
 10  T_Number               18062 non-null  float64
 11  CCL_cm                 18062 non-null  float64
 12  CCW_cm                 18035 non-null  float64
 13  Weight_Kg              12653 non-null  float64
 14  Sex                    13732 non-null  object 
 15  Tu

In [73]:
#Impute NaN CCL_cm values, setting all of them as median
CCL_cm_median = df['CCL_cm'].median()
df['CCL_cm'].fillna(CCL_cm_median, inplace=True) 
df.isnull().sum()

Rescue_ID                   0
Date_TimeCaught             0
Researcher                  0
CaptureSite                 0
ForagingGround              0
CaptureMethod               0
Fisher                      0
LandingSite                 0
Species                     0
Tag_2                       0
T_Number                    0
CCL_cm                      0
CCW_cm                     27
Weight_Kg                5409
Sex                      4330
TurtleCharacteristics      52
Status                   3633
ReleaseSite                75
Date_TimeRelease         6108
dtype: int64

In [21]:
df['Sex'].value_counts()

Sex
Unknown         13578
Female            113
Male               39
Not_Recorded        2
Name: count, dtype: int64

In [18]:
df.describe()

CCL_cm        CCW_cm     Weight_Kg
count  18038.000000  18035.000000  12653.000000
mean      43.090390     40.253904      9.850731
std       11.004251      9.933058      9.737378
min        2.000000      2.000000      0.020000
25%       36.330000     34.000000      5.000000
50%       41.300000     39.300000      7.500000
75%       47.000000     44.100000     10.800000
max      122.750000    106.000000    140.000000

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18062 entries, 0 to 18061
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Rescue_ID              18062 non-null  object 
 1   Date_TimeCaught        18062 non-null  object 
 2   Researcher             18062 non-null  object 
 3   CaptureSite            18062 non-null  object 
 4   ForagingGround         18062 non-null  object 
 5   CaptureMethod          18062 non-null  object 
 6   Fisher                 18062 non-null  object 
 7   LandingSite            18062 non-null  object 
 8   Species                18062 non-null  object 
 9   Tag_2                  18062 non-null  int64  
 10  T_Number               18062 non-null  float64
 11  CCL_cm                 18062 non-null  float64
 12  CCW_cm                 18035 non-null  float64
 13  Weight_Kg              12653 non-null  float64
 14  Sex                    13732 non-null  object 
 15  Tu

In [39]:
# categorical_columns = ['LandingSite', 'Species', 'Tag_1', 'Tag_2',
                       # 'Lost_Tags', 'T_Number'] 

# for col in categorical_columns:
    # df[col] = df[col].astype('category')

In [40]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18062 entries, 0 to 18061
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Rescue_ID              18062 non-null  object  
 1   Date_TimeCaught        18062 non-null  object  
 2   Researcher             18062 non-null  object  
 3   CaptureSite            18062 non-null  object  
 4   ForagingGround         18062 non-null  object  
 5   CaptureMethod          18062 non-null  object  
 6   Fisher                 18062 non-null  object  
 7   LandingSite            18062 non-null  category
 8   Species                18062 non-null  category
 9   Tag_1                  18062 non-null  category
 10  Tag_2                  18062 non-null  category
 11  Lost_Tags              18062 non-null  category
 12  T_Number               18062 non-null  category
 13  CCL_cm                 18062 non-null  float64 
 14  CCW_cm                 18035 non-null 

In [55]:
df.value_counts()

Series([], Name: count, dtype: int64)

In [80]:
df.nunique()

Rescue_ID                18062
Date_TimeCaught           5237
Researcher                  35
CaptureSite                 29
ForagingGround               3
CaptureMethod               15
Fisher                    2085
LandingSite                  5
Species                      8
Tag_2                        2
T_Number                     2
CCL_cm                    1338
CCW_cm                    1262
Weight_Kg                 1937
Sex                          4
TurtleCharacteristics    16342
Status                     439
ReleaseSite                271
Date_TimeRelease          3008
dtype: int64

In [81]:
df.head(15)

Rescue_ID Date_TimeCaught     Researcher    CaptureSite ForagingGround  \
0   2000_RE_0060      2000-12-22  Researcher_25  CaptureSite_0          Ocean   
1   2001_RE_0187      2001-10-28   Researcher_6  CaptureSite_0          Ocean   
2   2001_RE_0197      2001-11-01   Researcher_6  CaptureSite_0          Ocean   
3   2002_RE_0031      2002-03-11  Researcher_32  CaptureSite_0          Ocean   
4   2002_RE_0118      2002-08-08  Researcher_25  CaptureSite_0          Ocean   
5   2002_RE_0119      2002-08-10  Researcher_25  CaptureSite_0          Ocean   
6   2002_RE_0214      2002-10-21  Researcher_25  CaptureSite_0          Ocean   
7   2002_RE_0215      2002-10-21  Researcher_25  CaptureSite_0          Ocean   
8   2002_RE_0218      2002-10-22  Researcher_30  CaptureSite_0          Ocean   
9   2003_RE_0187      2003-06-09  Researcher_32  CaptureSite_0          Ocean   
10  2003_RE_0322      2003-08-11  Researcher_25  CaptureSite_0          Ocean   
11  2003_RE_0363      2003-09-02  Researcher_32  CaptureSite_0          Ocean   
12  2003_RE_0411      2003-09-22  Researcher_30  CaptureSite_0          Ocean   
13  2003_RE_0412      2003-09-22  Researcher_30  CaptureSite_0          Ocean   
14  2003_RE_0413      2003-09-22  Researcher_30  CaptureSite_0          Ocean   

   CaptureMethod       Fisher                        LandingSite    Species  \
0            Net  Fisher_1072  LandingSite_CaptureSiteCategory_2  Species_6   
1            Net   Fisher_520  LandingSite_CaptureSiteCategory_2  Species_6   
2            Net  Fisher_1669  LandingSite_CaptureSiteCategory_2  Species_5   
3            Net  Fisher_1798  LandingSite_CaptureSiteCategory_2  Species_6   
4        Beached  Fisher_1918  LandingSite_CaptureSiteCategory_2  Species_5   
5   Not_Recorded  Fisher_1918  LandingSite_CaptureSiteCategory_2  Species_5   
6            Net  Fisher_2013  LandingSite_CaptureSiteCategory_2  Species_6   
7            Net  Fisher_1815  LandingSite_CaptureSiteCategory_2  Species_6   
8            Net  Fisher_1815  LandingSite_CaptureSiteCategory_2  Species_5   
9            Net  Fisher_1066  LandingSite_CaptureSiteCategory_2  Species_5   
10           Net  Fisher_1114  LandingSite_CaptureSiteCategory_2  Species_5   
11      Longline  Fisher_2065  LandingSite_CaptureSiteCategory_2  Species_5   
12           Net  Fisher_2013  LandingSite_CaptureSiteCategory_2  Species_5   
13           Net  Fisher_2013  LandingSite_CaptureSiteCategory_2  Species_5   
14           Net  Fisher_2013  LandingSite_CaptureSiteCategory_2  Species_5   

    Tag_2  T_Number  CCL_cm  CCW_cm  Weight_Kg      Sex  \
0       0       0.0   64.70   62.60        NaN  Unknown   
1       0       0.0   35.85   31.35        NaN  Unknown   
2       0       0.0   51.80   49.20        NaN  Unknown   
3       0       0.0   60.50   59.00        NaN  Unknown   
4       0       0.0   34.70   33.00        NaN  Unknown   
5       0       0.0   33.20   30.70        NaN  Unknown   
6       0       0.0   37.40   33.50        NaN  Unknown   
7       0       0.0   40.30   37.20        NaN  Unknown   
8       0       0.0   44.00   43.20        NaN  Unknown   
9       0       0.0   48.30   43.30        NaN  Unknown   
10      0       0.0   43.30   43.60        NaN  Unknown   
11      0       0.0   43.40   41.70        NaN  Unknown   
12      0       0.0   58.20   53.30        NaN  Unknown   
13      0       0.0   40.20   39.30        NaN  Unknown   
14      0       0.0   52.50   47.50        NaN  Unknown   

                                                                                                                                                                                                                                              TurtleCharacteristics  \
0                                                                                                                                                                                                                                            

In [82]:
# Create dummy variables 
df_dummies = pd.get_dummies(df, columns=['LandingSite', 'Species'], drop_first=True) 

df_dummies

Rescue_ID Date_TimeCaught     Researcher    CaptureSite  \
0      2000_RE_0060      2000-12-22  Researcher_25  CaptureSite_0   
1      2001_RE_0187      2001-10-28   Researcher_6  CaptureSite_0   
2      2001_RE_0197      2001-11-01   Researcher_6  CaptureSite_0   
3      2002_RE_0031      2002-03-11  Researcher_32  CaptureSite_0   
4      2002_RE_0118      2002-08-08  Researcher_25  CaptureSite_0   
...             ...             ...            ...            ...   
18057  2018_RE_1511      2018-12-18  Researcher_30  CaptureSite_9   
18058  2018_RE_1514      2018-12-18  Researcher_30  CaptureSite_9   
18059  2018_RE_1532      2018-12-24  Researcher_30  CaptureSite_9   
18060  2018_RE_1533      2018-12-24  Researcher_30  CaptureSite_9   
18061  2018_RE_1550      2018-12-28  Researcher_30  CaptureSite_9   

      ForagingGround CaptureMethod       Fisher  Tag_2  T_Number  CCL_cm  ...  \
0              Ocean           Net  Fisher_1072      0       0.0   64.70  ...   
1              Ocean           Net   Fisher_520      0       0.0   35.85  ...   
2              Ocean           Net  Fisher_1669      0       0.0   51.80  ...   
3              Ocean           Net  Fisher_1798      0       0.0   60.50  ...   
4              Ocean       Beached  Fisher_1918      0       0.0   34.70  ...   
...              ...           ...          ...    ...       ...     ...  ...   
18057          Ocean           Net   Fisher_569      0       0.0   57.13  ...   
18058          Ocean           Net   Fisher_125      0       0.0   42.07  ...   
18059          Ocean           Net  Fisher_1343      0       0.0   57.20  ...   
18060          Ocean           Net  Fisher_1551      0       0.0   51.90  ...   
18061          Ocean           Net  Fisher_1551      1       0.0   34.60  ...   

       LandingSite_LandingSite_CaptureSiteCategory_2  \
0                                               True   
1                                               True   
2                                               True   
3                                               True   
4                                               True   
...                                              ...   
18057                                          False   
18058                                          False   
18059                                          False   
18060                                          False   
18061                                          False   

       LandingSite_LandingSite_CaptureSiteCategory_3  \
0                                              False   
1                                              False   
2                                              False   
3                                              False   
4                                              False   
...                                              ...   
18057                                          False   
18058                                          False   
18059                                          False   
18060                                          False   
18061                                          False   

      LandingSite_LandingSite_CaptureSiteCategory_4 Species_Species_1  \
0                                             False             False   
1                                             False             False   
2                                             False             False   
3                                             False             False   
4                                             False             False   
...                                             ...               ...   
18057                                         False             False   
18058                                         False             False   
18059                                         False             False   
18060                                         False             False   
18061                                         False        

In [83]:
print((df == 0).sum())

Rescue_ID                    0
Date_TimeCaught              0
Researcher                   0
CaptureSite                  0
ForagingGround               0
CaptureMethod                0
Fisher                       0
LandingSite                  0
Species                      0
Tag_2                    13151
T_Number                 18024
CCL_cm                       0
CCW_cm                       0
Weight_Kg                    0
Sex                          0
TurtleCharacteristics        0
Status                       0
ReleaseSite                  0
Date_TimeRelease             0
dtype: int64


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18062 entries, 0 to 18061
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Rescue_ID              18062 non-null  object 
 1   Date_TimeCaught        18062 non-null  object 
 2   Researcher             18062 non-null  object 
 3   CaptureSite            18062 non-null  object 
 4   ForagingGround         18062 non-null  object 
 5   CaptureMethod          18062 non-null  object 
 6   Fisher                 18062 non-null  object 
 7   LandingSite            18062 non-null  object 
 8   Species                18062 non-null  object 
 9   Tag_2                  18062 non-null  int64  
 10  T_Number               18062 non-null  float64
 11  CCL_cm                 18062 non-null  float64
 12  CCW_cm                 18035 non-null  float64
 13  Weight_Kg              12653 non-null  float64
 14  Sex                    13732 non-null  object 
 15  Tu

In [85]:
df.nunique() 

Rescue_ID                18062
Date_TimeCaught           5237
Researcher                  35
CaptureSite                 29
ForagingGround               3
CaptureMethod               15
Fisher                    2085
LandingSite                  5
Species                      8
Tag_2                        2
T_Number                     2
CCL_cm                    1338
CCW_cm                    1262
Weight_Kg                 1937
Sex                          4
TurtleCharacteristics    16342
Status                     439
ReleaseSite                271
Date_TimeRelease          3008
dtype: int64

In [84]:
df['Status'].unique()

array(['Released', ' 2smb 3LLS& 1sm b R supra',
       ' b scar 4RMS& 2LLS& 4LLS',
       ' sm patches of calcerous algae on shell and supras',
       ' b join 3/4 LLS', nan, 'Admitted', ' chips on RM',
       ' tip of RS CS missing',
       " quite large piece missing from 1 + 2 scale area on LFF - possibly old tagging+ but unlikley that tagged there and piece is large+ maybe old wound.  a lot of small b's on flesh",
       ' green algae on scutes',
       ' no other major physical characteristics',
       ' light green algae on scutes', ' no other major characteristics',
       ' 6 LLS& small notch in the joints between both 10th and 11th MS',
       ' B scar on 1st CS', ' notch on LRF',
       ' few small barnacles on carapace', ' barnacles on ???',
       ' tiny barnacles on plastron', ' green algae on rear carapace',
       ' small barnacles on neck', ' small barnacles on flippers',
       ' RSCS tip missing', ' green algae on carapace',
       ' 7+ 9 + 10 LM scutes have some chip

In [13]:
df['Status'].nunique()

439

In [14]:
df.duplicated().value_counts()

False    18062
Name: count, dtype: int64